# Algoritmo - Inspeção Bilhetagem (2023)
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Carregamento e inSpeção de informações da bilhetagem (xml)_

# 1.0  Leitura da Base

## 1.1 Bilhetagem (Tratamento Inicial)

In [ ]:
# Carregando bibliotecas e plugins necessários 
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
# Parse do XML
tree = ET.parse('V20231107.xml')
root = tree.getroot()

In [ ]:
# Lista para armazenar os dados
data = []

# Conjunto para rastrear as passagens já adicionadas
passageiros_adicionados = set()

# Iterando sobre os elementos do XML
for movimento_diario in root.findall('.//MovimentoDiario'):
    for empresa in movimento_diario.findall('.//Empresa'):
        veiculo = empresa.find('.//Veiculo')
        veiculo_data = {
            'veiculo_Numero': veiculo.get('Numero') if veiculo is not None else None,
            'veiculo_validador': veiculo.get('validador') if veiculo is not None else None
        }
        
        for linha in empresa.findall('.//Linha'):
            linha_data = {
                'linha_Numero': linha.get('Numero'),
                'linha_jornada': linha.get('jornada'),
                'linha_num_operador': linha.get('num_operador'),
                'linha_tabela': linha.get('tabela'),
                'linha_hora_abertura': linha.get('hora_abertura'),
                'linha_hora_fechamento': linha.get('hora_fechamento'),
                'veiculo_Numero': veiculo_data['veiculo_Numero'],
                'veiculo_validador': veiculo_data['veiculo_validador']
            }

            for viagem in linha.findall('.//Viagem'):
                for passageiro in viagem.findall('.//Passageiro'):
                    passageiro_data = (
                        passageiro.get('data_hora'),
                        passageiro.get('integracao_bum'),
                        passageiro.get('valor_subsidio'),
                        passageiro.get('evento'),
                        passageiro.get('sigben'),
                        passageiro.get('integracao'),
                        passageiro.get('valor_pago'),
                        passageiro.get('tipo'),
                        passageiro.get('Matricula'),
                        passageiro.get('valor_repasse_metro'),
                        passageiro.get('latitude'),
                        passageiro.get('longitude'),
                        linha.get('Numero'),
                        linha.get('jornada'),
                        linha.get('num_operador'),
                        linha.get('tabela'),
                        linha.get('hora_abertura'),
                        linha.get('hora_fechamento'),
                        veiculo_data['veiculo_Numero'],
                        veiculo_data['veiculo_validador']
                    )
                    if passageiro_data not in passageiros_adicionados:
                        passageiros_adicionados.add(passageiro_data)
                        data.append({
                            'passageiro_data_hora': passageiro.get('data_hora'),
                            'passageiro_integracao_bum': passageiro.get('integracao_bum'),
                            'passageiro_valor_subsidio': passageiro.get('valor_subsidio'),
                            'passageiro_evento': passageiro.get('evento'),
                            'passageiro_sigben': passageiro.get('sigben'),
                            'passageiro_integracao': passageiro.get('integracao'),
                            'passageiro_valor_pago': passageiro.get('valor_pago'),
                            'passageiro_tipo': passageiro.get('tipo'),
                            'passageiro_Matricula': passageiro.get('Matricula'),
                            'passageiro_valor_repasse_metro': passageiro.get('valor_repasse_metro'),
                            'passageiro_latitude': passageiro.get('latitude'),
                            'passageiro_longitude': passageiro.get('longitude'),
                            'linha_Numero': linha.get('Numero'),
                            'linha_jornada': linha.get('jornada'),
                            'linha_num_operador': linha.get('num_operador'),
                            'linha_tabela': linha.get('tabela'),
                            'linha_hora_abertura': linha.get('hora_abertura'),
                            'linha_hora_fechamento': linha.get('hora_fechamento'),
                            'veiculo_Numero': veiculo_data['veiculo_Numero'],
                            'veiculo_validador': veiculo_data['veiculo_validador']
                        })

In [ ]:
# Criando df
bilhetagem = pd.DataFrame(data)

In [ ]:
bilhetagem

In [ ]:
# Convertendo inicialmente data para manipulação 
bilhetagem['passageiro_data_hora'] = pd.to_datetime(bilhetagem['passageiro_data_hora'])

In [ ]:
# Verificando dados nulos 
bilhetagem.isnull().sum()

In [ ]:
''' OU SEJA, NÃO HÁ INFORMAÇÕES NULAS DE NENHUMA VALIDAÇÃO E NEM DE DADOS DOS VEICULOS E DAS LIHHAS UTILIZADAS PELOS USUARIOS! '''

## 1.2 Inspeção inicial

In [ ]:
# Informações da base
bilhetagem.info()

In [ ]:
''' POR INSPEÇÃO FOI CONSTATADO QUE EXISTEM DADOS DE LAT E LONG VAZIOS!'''

In [ ]:
# Filtrando as linhas do DATAFRAME onde a coluna passageiro_latitude tem valores vazios
lat_long_vazios = bilhetagem[bilhetagem['passageiro_latitude'] == '']
lat_long_vazios

In [ ]:
# VERIFICANDO AS LINHAS DE ONIBUS DOS DADOS SEM INFORMAÇÕES GEOMETRICAS 
lat_long_vazios['linha_Numero'].unique()

In [ ]:
''' SE DEVE TER UMA OU MAIS ABORDAGENS DE GEORREFERENCIAMENTO DESSAS VALIDAÇÕES SEM LAT E LONG! '''

## 1.3 Verificação de validações em trecho de interesse
    ''' como uma parcela das validações tem valores de latitude e longitude vazios, a princípio vamos analisar somente aqueles que têm essas informações presentes na base! '''

In [ ]:
# Filtrando somente os dados que tem informações gemetricas 
bilhetagem_geo = bilhetagem[~bilhetagem['passageiro_latitude'].isin(lat_long_vazios['passageiro_latitude'])]

In [ ]:
bilhetagem_geo

In [ ]:
# Convertendo dados de lat e long para float 
bilhetagem_geo['passageiro_latitude'] = bilhetagem_geo['passageiro_latitude'].astype('float')
bilhetagem_geo['passageiro_longitude'] = bilhetagem_geo['passageiro_longitude'].astype('float')

In [ ]:
# FILTRANDO DADOS DO VECIULO EM TRECHO ESPECIFICO
bilhetagem_trecho = bilhetagem_geo[(bilhetagem_geo.passageiro_latitude>=-3.73667)
    & (bilhetagem_geo.passageiro_longitude>=-38.56944) & (bilhetagem_geo.passageiro_longitude<=-38.54167)
                                & (bilhetagem_geo.passageiro_latitude<=-3.73083)]

In [ ]:
bilhetagem_trecho

In [ ]:
# LINHAS NESSA REGIÃO
bilhetagem_trecho['linha_Numero'].unique()

In [ ]:
# filtrando dados de linhas especifica 
linha = bilhetagem_trecho[bilhetagem_trecho.linha_Numero=='28']

In [ ]:
# Veiculos da linha filtrada que tiveram validações nessa região
linha['veiculo_Numero'].unique()

In [ ]:
linha # Validações da linha filtrada!

In [ ]:
# FILTRANDO HORARIO E VEICULO ESPECIFICO
linha_horario = linha[(linha.veiculo_Numero=='35025') & (linha.passageiro_data_hora>='2023-11-06 06:40:00')
                      & (linha.passageiro_data_hora<='2023-11-06 07:00:00')]

In [ ]:
linha_horario

In [ ]:
lat = linha_horario['passageiro_latitude'].mean()
lon = linha_horario['passageiro_longitude'].mean()

In [ ]:
# Criando um mapa centrado em FORTALEZA
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[lat, lon], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in linha_horario.iterrows():
    folium.CircleMarker(location=[row['passageiro_latitude'], row['passageiro_longitude']],
                        radius=10,
                        color='blue',
                        fill=True,
                        fill_color='green',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['passageiro_data_hora'], ''), max_width=3)).add_to(marker_cluster)

In [ ]:
mapa

# ------------------------------------------------------------------------